In [1]:
import collections
%matplotlib inline
import matplotlib
import numpy as np
import openml
import pandas as pd
import time
print(openml.config.server)

https://www.openml.org/api/v1/xml


In [2]:
automlbench_dids = [41021, 42225, 42571, 4549, 42572, 42570, 42705,
                   42728, 550, 546, 541, 507, 505, 287, 216, 41540,
                   42688, 422, 416, 42724, 42727, 42729, 42726, 42730,
                   201, 41980, 42731, 531, 42563, 574, 3050, 3277, 43071]

In [3]:
all_datasets = openml.datasets.list_datasets()
to_consider = openml.datasets.list_datasets()
TOTAL = len(all_datasets)
# print(TOTAL)
# # all_tasks = openml.tasks.list_tasks()
to_remove = set()
# # print(len(all_tasks))
# print("From:", len(to_consider)) # 3004

In [4]:
# remove datasets from the datasets list which are in the automl bench or close relatives
print("From:", len(to_consider))

# remove exact duplicates
for did in all_datasets:
    if did in automlbench_dids:
        to_remove.add(did)

for did in to_remove:
    if did in to_consider:
        del to_consider[did]
assert len(to_consider) == len(all_datasets) - len(automlbench_dids)

print("To:", len(to_consider)) # 2965

From: 5180
To: 5147


In [5]:
# Remove datasets with too few features, instances or classes
print("From:", len(to_consider))

for did in all_datasets:
    try:
        if all_datasets[did]['NumberOfInstances'] * all_datasets[did]['NumberOfFeatures'] < 1_000:
            to_remove.add(did)
        elif all_datasets[did]['NumberOfInstances'] >= 500_000:
            to_remove.add(did)
        elif all_datasets[did]['NumberOfInstances'] < 20:
            to_remove.add(did)
        elif all_datasets[did]['NumberOfFeatures'] < 2:
            to_remove.add(did)
        elif all_datasets[did]['NumberOfFeatures'] > 2500:
            to_remove.add(did)
        elif all_datasets[did]['NumberOfClasses'] > 1:
            to_remove.add(did)
        elif all_datasets[did]['NumberOfInstances'] * all_datasets[did]['NumberOfFeatures'] > 10_000_000:
            to_remove.add(did)
    except KeyError:
        #print(did, all_datasets[did])
        to_remove.add(did)
        
for did in to_remove:
    if did in to_consider:
        del to_consider[did]
        
print("To:", len(to_consider)) # 651

From: 5147
To: 1547


In [6]:
all_datasets_as_frame = pd.DataFrame.from_dict(all_datasets, orient='index')

In [7]:
# Remove all datasets with "BNG" in name
print("From:", len(to_consider))

for did in to_consider:
    if to_consider[did]['name'].startswith("BNG"):
        to_remove.add(did)
        
for did in to_remove:
    if did in to_consider:
        del to_consider[did]
        
print("To:", len(to_consider)) # 641

From: 1547
To: 1540


In [8]:
# check for similar names and manually check overlaps
# def return_triplets(name):
#     triplets = set()
#     name = name.lower()
#     for i in range(len(name) - 2):
#         triplets.add(name[i: i+3])
#     return triplets

# manually_look_at = []
# for did in to_consider:
#     for did2 in automlbench_dids:
#         automl_triplets = return_triplets(all_datasets[did2]['name'])
#         other_triplets = return_triplets(all_datasets[did]['name'])
#         n_hits = sum([trip in other_triplets for trip in automl_triplets])
#         ratio = n_hits / len(automl_triplets)
#         if n_hits > 2:
#             manually_look_at.append([n_hits, ratio, all_datasets[did]['name'], all_datasets[did2]['name'], did, did2])

# df = pd.DataFrame(manually_look_at)
# with open("Manualautomlregr.csv", "w") as fh:
#    fh.write(df.to_csv())     


In [9]:
# This list contains all manually collected dataset ids as in 
# https://docs.google.com/spreadsheets/d/1XH1mQFq7-1fs28F-CQJo6i6Wg7IlpDS0aDTAJBoaD7k/edit#gid=0
print("From:", len(to_consider))
for did in [209, 543, 4545, 42092, 42635, 42723, 43465, 43984, 43986, 
            43989, 43992, 43994, 43996, 43997, 43998, 43999, 44002, 
            44009, 44011, 44013, 44014, 44015, 44016, 44019, 44039, 
            44042, 44044, 44047, 44049, 44051, 44054, 44057, 44059, 
            44061, 44062, 44063, 44064, 44066, 44134, 44136, 44138, 
            44139, 44140, 4414, 44142, 44144, 44152, 44201, 44202, 
            44203, 44269, 44956, 44979, 44985, 44987, 44988, 44990, 
            45031, 45032, 45033, 45041, 45042, 42713, 44000, 44017, 44048, 44063, 44142]:
    to_remove.add(did)
 
for did in to_remove:
    if did in to_consider:
        del to_consider[did]
print("To:", len(to_consider)) # 603

From: 1540
To: 1475


In [10]:
# # check for similar metafeatures and manually check overlaps
# metafeature_names = [
#     'MaxNominalAttDistinctValues', 'NumberOfClasses',
#     'NumberOfFeatures', 'NumberOfInstances', 'NumberOfInstancesWithMissingValues', 'NumberOfMissingValues',
#     'NumberOfNumericFeatures', 'NumberOfSymbolicFeatures',
# ]

# simple_metafeatures_to_consider = pd.DataFrame(
#     {k: v for k, v in all_datasets.items() if k in to_consider}
# ).transpose()[metafeature_names]
# simple_metafeatures_automl = pd.DataFrame(
#     {k: v for k, v in all_datasets.items() if k in automlbench_dids}
# ).transpose()[metafeature_names]

# # Checking the hamming distance of the datasets
# manually_look_at = []
# for did1, mf1 in simple_metafeatures_to_consider.iterrows():
#     for did2, mf2 in simple_metafeatures_automl.iterrows():
#         n_hits = np.sum(mf1 == mf2)
#         ratio = n_hits / len(mf1)
#         n_hits = np.sum(
#             mf1[['NumberOfFeatures', 'NumberOfInstances']] == mf2[['NumberOfFeatures', 'NumberOfInstances']]
#         )
#         if ratio > 0.5 or n_hits == 2:
#             entry = [ratio, n_hits, all_datasets[did1]['name'], all_datasets[did2]['name'], did1, did2]
#             mfs = np.array([[mf1[n], mf2[n]] for n in metafeature_names]).flatten()
#             entry.extend(mfs)
#             manually_look_at.append(entry)

# header = ["ratio", "n_hits", "name", "automl name", "did", "automl did"] 
# mfs = list(np.array([[m, m] for m in metafeature_names]).flatten())
# df = pd.DataFrame(manually_look_at, columns = header + mfs)
# # with open("ManualMetafeaturesregr.csv", "w") as fh:
# #    fh.write(df.to_csv())            

In [11]:
# This list contains all manually collected dataset ids as in 
# https://docs.google.com/spreadsheets/d/1SWEBsFTykdUMq-wBcx9NHchhdolB632EJYaAmQwwQtM/edit#gid=0
print("From:", len(to_consider))
for did in [315, 580, 610, 652, 41938, 41968, 41969, 43070, 43672, 43888, 43983, 43991,
            44008, 44133, 44141, 44971, 44972, 44989, 45030]:
    to_remove.add(did)
 
for did in to_remove:
    if did in to_consider:
        del to_consider[did]
print("To:", len(to_consider)) # 599
# check whether we can work usefully with creditcard (1597)

From: 1475
To: 1456


In [12]:
pd.set_option('display.max_rows', 25)
columns = ['name', 'NumberOfClasses', 'NumberOfFeatures', 'NumberOfInstances', ]

to_consider_stats = pd.DataFrame(
    {k: v for k, v in all_datasets.items() if k in to_consider}
).transpose()[columns]

to_consider_stats

,name,NumberOfClasses,NumberOfFeatures,NumberOfInstances
8,liver-disorders,0.0,6.0,345.0
189,kin8nm,0.0,9.0,8192.0
191,wisconsin,0.0,33.0,194.0
194,cleveland,0.0,14.0,303.0
195,auto_price,0.0,16.0,159.0
...,...,...,...,...
45076,churn,0.0,21.0,5000.0
45078,nomao,0.0,119.0,34465.0
45079,road-safety,0.0,33.0,111762.0
45081,Tallo,0.0,21.0,307014.0


In [13]:
# remove sparse datasets
# print("From:", len(to_consider))
# start = time.time()
# for i, did in enumerate(to_consider):
#     #print(did)
#     if (i+1) % 25 == 0:
#         print("%4.2fsec: %d/%d" % (time.time() - start, i+1, len(to_consider)))
#     d = openml.datasets.get_dataset(int(did), download_data=False)
#     #print(
#     #    d.format.lower(), 
#     #    'sparse' in d.format.lower(), 
#     #    d.description and 'CLASSINDEX: none specific' in d.description
#     #)
#     if 'sparse' in d.format.lower():
#         to_remove.add(did)
#     if d.description and 'CLASSINDEX: none specific' in d.description:
#         to_remove.add(did)
#     for f in d.features.values():
#         if f.data_type == 'string':
#             to_remove.add(did)

# for did in to_remove:
#     if did in to_consider:
#         del to_consider[did]
# print("To:", len(to_consider)) # 599

In [14]:
to_consider

OrderedDict([(8,
              {'did': 8,
               'name': 'liver-disorders',
               'version': 1,
               'uploader': '1',
               'status': 'active',
               'format': 'ARFF',
               'NumberOfClasses': 0.0,
               'NumberOfFeatures': 6.0,
               'NumberOfInstances': 345.0,
               'NumberOfInstancesWithMissingValues': 0.0,
               'NumberOfMissingValues': 0.0,
               'NumberOfNumericFeatures': 6.0,
               'NumberOfSymbolicFeatures': 0.0}),
             (189,
              {'did': 189,
               'name': 'kin8nm',
               'version': 1,
               'uploader': '1',
               'status': 'active',
               'format': 'ARFF',
               'NumberOfClasses': 0.0,
               'NumberOfFeatures': 9.0,
               'NumberOfInstances': 8192.0,
               'NumberOfInstancesWithMissingValues': 0.0,
               'NumberOfMissingValues': 0.0,
               'NumberOfNumeric

In [15]:
# # remove identical/similar datasets (to reduce the load computing the meta-data)
# # check for similar names and manually check overlaps
# def return_triplets(name):
#     triplets = set()
#     name = name.lower()
#     for i in range(len(name) - 2):
#         triplets.add(name[i: i+3])
#     return triplets

# #Remove QSAR datasets

# manually_look_at = []
# for i, did in enumerate(to_consider):
#     for j, did2 in enumerate(to_consider):
        
#         if all_datasets[did]["name"].startswith("QSAR") or all_datasets[did2]["name"].startswith("QSAR"):
#             continue
#         if j <= i:
#             continue
#         if len(all_datasets[did2]['name']) < 3:
#             continue
#         other2_triplets = return_triplets(all_datasets[did2]['name'])
#         other_triplets = return_triplets(all_datasets[did]['name'])
#         n_hits = sum([trip in other_triplets for trip in other2_triplets])
#         ratio = n_hits / len(other2_triplets)
#         if n_hits > 2:
#             manually_look_at.append([n_hits, ratio, all_datasets[did]['name'], all_datasets[did2]['name'], did, did2])

# df = pd.DataFrame(manually_look_at)
# with open("manual_check.csv", "w") as fh:
#    fh.write(df.to_csv())   

In [17]:
#filter all_datasets as frame such that the name contains IEEE80211aa-GATS
pd.set_option('display.max_rows', 87)
all_datasets_as_frame[all_datasets_as_frame['name'].str.startswith("Meta_Album")]

,did,name,version,uploader,status,format,MajorityClassSize,MaxNominalAttDistinctValues,MinorityClassSize,NumberOfClasses,NumberOfFeatures,NumberOfInstances,NumberOfInstancesWithMissingValues,NumberOfMissingValues,NumberOfNumericFeatures,NumberOfSymbolicFeatures
44237,44237,Meta_Album_BCT_Micro,1,30980,active,arff,40.0,NaN,40.0,20.0,3.0,800.0,800.0,800.0,1.0,0.0
44238,44238,Meta_Album_PLK_Micro,1,30980,active,arff,40.0,NaN,40.0,20.0,3.0,800.0,800.0,800.0,1.0,0.0
44239,44239,Meta_Album_FLW_Micro,1,30980,active,arff,40.0,NaN,40.0,20.0,3.0,800.0,800.0,800.0,1.0,0.0
44240,44240,Meta_Album_SPT_Micro,1,30980,active,arff,40.0,NaN,40.0,20.0,3.0,800.0,800.0,800.0,1.0,0.0
44241,44241,Meta_Album_BRD_Micro,1,30980,active,arff,40.0,NaN,40.0,20.0,3.0,800.0,800.0,800.0,1.0,0.0
44242,44242,Meta_Album_PLT_VIL_Micro,1,30980,active,arff,40.0,NaN,40.0,20.0,3.0,800.0,0.0,0.0,0.0,0.0
44243,44243,Meta_Album_MD_MIX_Micro,1,30980,active,arff,NaN,NaN,NaN,0.0,69.0,800.0,800.0,18840.0,46.0,1.0
44244,44244,Meta_Album_TEX_Micro,1,30980,active,arff,40.0,NaN,40.0,20.0,3.0,800.0,800.0,800.0,1.0,0.0
44245,44245,Meta_Album_CRS_Micro,1,30980,active,arff,40.0,NaN,40.0,20.0,3.0,800.0,800.0,800.0,1.0,0.0
44246,44246,Meta_Album_RESISC_Micro,1,30980,active,arff,40.0,NaN,40.0,20.0,3.0,800.0,800.0,800.0,1.0,0.0


In [18]:
45028

45028

In [19]:
all_datasets_as_frame[all_datasets_as_frame['name'].str.lower().str.contains("churn")]

,did,name,version,uploader,status,format,MajorityClassSize,MaxNominalAttDistinctValues,MinorityClassSize,NumberOfClasses,NumberOfFeatures,NumberOfInstances,NumberOfInstancesWithMissingValues,NumberOfMissingValues,NumberOfNumericFeatures,NumberOfSymbolicFeatures
1112,1112,KDDCup09_churn,1,2,active,ARFF,46328.0,15415.0,3672.0,2.0,231.0,50000.0,50000.0,8024152.0,192.0,39.0
40701,40701,churn,1,869,active,ARFF,4293.0,10.0,707.0,2.0,21.0,5000.0,0.0,0.0,16.0,5.0
42178,42178,telco-customer-churn,1,1140,active,arff,5174.0,NaN,1869.0,2.0,20.0,7043.0,0.0,0.0,3.0,0.0
42758,42758,KDDCup09-Churn,1,11601,active,arff,46328.0,NaN,3672.0,2.0,15001.0,50000.0,50000.0,25108569.0,14811.0,1.0
42775,42775,KDDCup09-Churn,2,11601,active,arff,46328.0,NaN,3672.0,2.0,15001.0,50000.0,50000.0,14616450.0,14740.0,261.0
43390,43390,Churn-for-Bank-Customers,1,30126,active,arff,NaN,NaN,NaN,NaN,14.0,10000.0,0.0,0.0,11.0,0.0
44226,44226,NewspaperChurn,1,30294,active,arff,12818.0,NaN,3037.0,2.0,19.0,15855.0,1394.0,1430.0,4.0,0.0
44227,44227,South_Asian_Churn_dataset,1,30294,active,arff,1000.0,NaN,1000.0,2.0,14.0,2000.0,279.0,453.0,9.0,0.0
44228,44228,Churn_Telco_Europa,1,30294,active,arff,NaN,NaN,NaN,0.0,20.0,190776.0,5621.0,22484.0,19.0,0.0
44229,44229,Churn_Telco_Europa,2,30294,active,arff,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [20]:
tmp_set = {44243, 44252, 44243, 44280, 44243, 44287, 44243,	44296, 44243,	44310, 44252,	44280, 44252,	44287, 44252,	44296, 44252,	44310, 44280,	44287, 44280,	44296, 44280,	44310, 44287,	44296, 44287,	44310, 44296,	44310} 

In [21]:
tmp_set

{44243, 44252, 44280, 44287, 44296, 44310}

In [22]:
all_datasets_as_frame

,did,name,version,uploader,status,format,MajorityClassSize,MaxNominalAttDistinctValues,MinorityClassSize,NumberOfClasses,NumberOfFeatures,NumberOfInstances,NumberOfInstancesWithMissingValues,NumberOfMissingValues,NumberOfNumericFeatures,NumberOfSymbolicFeatures
2,2,anneal,1,1,active,ARFF,684.0,7.0,8.0,5.0,39.0,898.0,898.0,22175.0,6.0,33.0
3,3,kr-vs-kp,1,1,active,ARFF,1669.0,3.0,1527.0,2.0,37.0,3196.0,0.0,0.0,0.0,37.0
4,4,labor,1,1,active,ARFF,37.0,3.0,20.0,2.0,17.0,57.0,56.0,326.0,8.0,9.0
5,5,arrhythmia,1,1,active,ARFF,245.0,13.0,2.0,13.0,280.0,452.0,384.0,408.0,206.0,74.0
6,6,letter,1,1,active,ARFF,813.0,26.0,734.0,26.0,17.0,20000.0,0.0,0.0,16.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
45269,45269,munin_6,1,33148,active,arff,NaN,NaN,NaN,NaN,1041.0,5000.0,0.0,0.0,0.0,0.0
45270,45270,munin_7,1,33148,active,arff,NaN,NaN,NaN,NaN,1041.0,5000.0,0.0,0.0,0.0,0.0
45271,45271,munin_8,1,33148,active,arff,NaN,NaN,NaN,NaN,1041.0,5000.0,0.0,0.0,0.0,0.0
45272,45272,munin_9,1,33148,active,arff,NaN,NaN,NaN,NaN,1041.0,5000.0,0.0,0.0,0.0,0.0


In [23]:
# This list contains all manually collected dataset ids as in 
# https://docs.google.com/spreadsheets/d/12Tmtgrz_92fuOKoIIIb6ROGkd65nv7R3g7EZAYwtTos/edit#gid=0
print("From:", len(to_consider))
for did in [45013, 45014, 44980, 195, 42372, 573, 43981, 43982, 43990,
            44007, 44132, 45029, 44978, 43882, 42900, 562, 43987, 43995, 44012, 44137,
            555, 556, 44055, 43127, 504, 568, 570, 575, 576, 577, 578, 670, 671, 672,
            44041, 44056, 43056, 1436, 1435, 44965, 44003, 44020, 23515,42436, 42437, 
            42438, 42439, 42444, 42445, 41514, 41515, 41516, 41517, 41518, 41519, 43384, 
            43483, 43582, 43943, 43959, 44187, 44191, 44192, 44194, 44223, 41516, 41517, 43123, 
            41523, 41524, 41525, 43127, 44236, 41928, 42131, 42559, 42720, 43928, 44004, 44021, 
            44146, 45048, 42559, 42720, 43881, 43882, 44235, 43127, 44068, 44069, 43144, 44069, 43871, 44967,
            42352, 44029, 44069, 44962, 43440, 42437, 42438, 42439, 42444, 42445, 45043, 42822, 43682, 43050,
            43056, 44957, 43079, 44055, 44005, 44022, 44147, 44983, 44006, 44023, 44148, 44964, 43180, 43190, 
            43192, 43194, 43198, 43256, 44179, 44180, 44214, 44218, 44219, 44220, 44221, 44267, 43483, 43582,
            43874, 43403, 43665, 43759, 43466, 43692, 44976, 43962, 44960, 44025, 43939, 43979, 44024, 44031, 
            44090, 44362, 44363, 44364, 44365, 44366, 44977, 45025, 44053, 44252, 44280, 44243, 44252, 44280, 
            44794, 44296, 44310, 45015] :
    to_remove.add(did)
 
for did in to_remove:
    if did in to_consider:
        del to_consider[did]
print("To:", len(to_consider)) # 503

From: 1456
To: 1319


In [24]:
# Subsample from 'topics', where there are dozens of super-similar
# datasots from one source
dataset_names = {did: value['name'] for did, value in all_datasets.items()
                 if did in to_consider}
# Set random number generator to always sample the same set
# of datasets
rng = np.random.RandomState(1)
# Friedman datasets
friedman_datasets = [
    did for did, name in dataset_names.items() if name.startswith('fri_c')
]
# print(friedman_datasets)
# ova & ap datasets
# ova_datasets = [
#     did for did, name in dataset_names.items() 
#     if name.startswith('OVA_') or name.startswith('AP_')
# ]

# print(ova_datasets)
# volcanoes_datasets = [
#     did for did, name in dataset_names.items() 
#     if name.startswith('volcanoes-')
# ]
# print(volcanoes_datasets)
# # AutoUniv
# auto_univ_datasets = [
#     did for did, name in dataset_names.items() 
#     if name.startswith('autoUniv-')
# ]
# print(auto_univ_datasets)

qsar_datasets =  [
    did for did, name in dataset_names.items() 
    if name.startswith('QSAR')
]

# print(qsar_datasets)
for dataset_ids_to_sample_from, num_keep in (
    (friedman_datasets, 10),
    # (ova_datasets, 7),
    # (volcanoes_datasets, 10),
    # (auto_univ_datasets, 4),
    (qsar_datasets, 50)
):
    choices_to_drop = np.random.choice(
        dataset_ids_to_sample_from,
        replace=False,
        size=len(dataset_ids_to_sample_from) - num_keep)
    for choice in choices_to_drop:
        if choice in to_consider:
            del to_consider[choice]
print("To:", len(to_consider)) # 454

To: 330


In [ ]:
# # check for similar metafeatures and manually check overlaps
# metafeature_names = [
#     'MajorityClassSize', 'MaxNominalAttDistinctValues', 'MinorityClassSize', 'NumberOfClasses',
#     'NumberOfFeatures', 'NumberOfInstances', 'NumberOfInstancesWithMissingValues', 'NumberOfMissingValues',
#     'NumberOfNumericFeatures', 'NumberOfSymbolicFeatures',
# ]

# simple_metafeatures_to_consider = pd.DataFrame(
#     {k: v for k, v in all_datasets.items() if k in to_consider}
# ).transpose()[metafeature_names]

# # Checking the hamming distance of the datasets
# manually_look_at = []
# for did1, mf1 in simple_metafeatures_to_consider.iterrows():
#     for did2, mf2 in simple_metafeatures_to_consider.iterrows():
#         if did2 <= did1:
#             continue
#         n_hits = np.sum(mf1 == mf2)
#         ratio = n_hits / len(mf1)
#         n_hits = np.sum(
#             mf1[['NumberOfFeatures', 'NumberOfInstances']] == mf2[['NumberOfFeatures', 'NumberOfInstances']]
#         )
#         if ratio > 0.5 or n_hits == 2:
#             entry = [ratio, n_hits, all_datasets[did1]['name'], all_datasets[did2]['name'], did1, did2]
#             mfs = np.array([[mf1[n], mf2[n]] for n in metafeature_names]).flatten()
#             entry.extend(mfs)
#             manually_look_at.append(entry)

# header = ["ratio", "n_hits", "name", "automl name", "did", "automl did"] 
# mfs = list(np.array([[m, m] for m in metafeature_names]).flatten())
# df = pd.DataFrame(manually_look_at, columns = header + mfs)
# with open("ManualMetafeaturesregression.csv", "w") as fh:
#    fh.write(df.to_csv()) 

In [25]:
# This list contains all manually collected dataset ids as in 
# https://docs.google.com/spreadsheets/d/1qVqH_ZVWJ3Eu0B6vXF0q2eQDMwiU5LXToN1L9ZfgLIs/edit#gid=0
print("From:", len(to_consider))
for did in [44026, 45055, 45052, 45053, 45054, 45055, 1027, 43988, 42111, 42112, 42113, 42361,
            1424, 1425, 1426, 1427, 1428, 1429,]:
    to_remove.add(did)
 
for did in to_remove:
    if did in to_consider:
        del to_consider[did]
print("To:", len(to_consider)) # 222

From: 330
To: 313


In [49]:

print("From:", len(to_consider))

for did in [
     43026, #single entry dataset
     43034, #single entry dataset
     1414, #bucket not accessible
     41275, #bucket not accessible
     45108, #bucket not accessible
     231, #below are all classification tasks with feature NumberofClasses 0
     298,
     301,
     516,
     524,
     703,
     1028,
     1097,
     1228,
     1430,
     1432,
     1433,
     1571,
     1572,
     1574,
     1575,
     1579,
     1589,
     1591,
     1593,
     4532,
     23395,
     41463,
     41943,
     42175,
     42176,
     42464,
     42636,
     42889,
     42931
]:
    to_remove.add(did)

# for did in [ # recreated samples from automl benchmark datasets
#     44593, 
#     44498, 
#     44557, 
#     44618, 
#     44780, 
#     44698, 
#     44729, 
#     44535
# ]:
#     to_remove.add(did)
    
for did in to_remove:
    if did in to_consider:
        del to_consider[did]
print("To:", len(to_consider)) # 209

From: 309
To: 278


In [50]:
final = pd.DataFrame(
    {k: v for k, v in all_datasets.items() if k in to_consider}
).transpose()
final.drop_duplicates(subset = ["name"], keep = "last", inplace = True)


In [48]:
for dataset_id in final.index:
    print("processing dataset_id: ", dataset_id)
    try: 
        dataset = openml.datasets.get_dataset(dataset_id)

    except Exception as e:
        print(dataset_id, f"Error: {e}")
        continue
    X, y, categorical_indicator, attribute_names = dataset.get_data(
        dataset_format='dataframe', target=dataset.default_target_attribute
    )
    if np.unique(y).shape[0] < 5:
        print(dataset_id, dataset.name, np.unique(y).shape[0])

processing dataset_id:  8
processing dataset_id:  189
processing dataset_id:  191
processing dataset_id:  194
processing dataset_id:  196
processing dataset_id:  197
processing dataset_id:  198
processing dataset_id:  203
processing dataset_id:  204
processing dataset_id:  206
processing dataset_id:  207
processing dataset_id:  213
processing dataset_id:  215
processing dataset_id:  217
processing dataset_id:  218
processing dataset_id:  222
processing dataset_id:  223
processing dataset_id:  224
processing dataset_id:  225
processing dataset_id:  227
processing dataset_id:  229
processing dataset_id:  230
processing dataset_id:  231
231 hungarian 2
processing dataset_id:  232
processing dataset_id:  294
processing dataset_id:  296
processing dataset_id:  298
298 coil2000 2
processing dataset_id:  299
processing dataset_id:  301
301 ozone_level 2
processing dataset_id:  308
processing dataset_id:  344
processing dataset_id:  405
processing dataset_id:  407
processing dataset_id:  408
p

processing dataset_id:  43308


processing dataset_id:  43351


processing dataset_id:  43379


43379 Pokmon-Legendary-Data 2
processing dataset_id:  43389


...

45017 Bioresponse 2
processing dataset_id:  45018


45018 electricity 2
processing dataset_id:  45040


45040 Intersectional-Bias-Assessment 2
processing dataset_id:  45056
45056 cmc 3
processing dataset_id:  45057
45057 jasmine 2
processing dataset_id:  45061
45061 dresses-sales 2
processing dataset_id:  45062
45062 shrutime 2
processing dataset_id:  45064
45064 Insurance 2
processing dataset_id:  45065
45065 bank-marketing 2
processing dataset_id:  45066
processing dataset_id:  45070
45070 kick 2
processing dataset_id:  45073
45073 eye_movements 2
processing dataset_id:  45074
45074 Shipping 2
processing dataset_id:  45075
45075 KDD 2
processing dataset_id:  45076
45076 churn 2
processing dataset_id:  45078
45078 nomao 2
processing dataset_id:  45079
45079 road-safety 2
processing dataset_id:  45081


processing dataset_id:  45108


45108 DutchTwitterDataset 3


In [ ]:
# dataset_ids = list(to_consider)
# datasets_to_tasks = collections.defaultdict(list)
# for task_id, task in all_tasks.items():
#     if task['ttid'] != 1:
#         continue
#     if task['estimation_procedure'] != '33% Holdout set':
#         continue
#     if task['did'] not in to_consider:
#         continue
#     datasets_to_tasks[task['did']].append(task_id)

In [ ]:
# Datasets which not yet have a 33% holdout task associated!
# datasets_without_task = sorted(list(set(to_consider) - set(datasets_to_tasks)))
# for dataset_wo_task in datasets_without_task:  
#     try:
#         d = openml.datasets.get_dataset(int(dataset_wo_task), download_data=False)            
#     except ValueError as e:
#         print(e)
#         continue
#     print(dataset_wo_task)

In [ ]:
# # Convert the 10-fold CV OpenML CC18 tasks into holdout tasks
# cc18_did_to_task_id = dict()
# for did in automlbench_dids:
#     if did not in cc18_did_to_task_id:
#         cc18_did_to_task_id[did] = list()
#     for task_id, task in all_tasks.items():
#         if task['ttid'] != 1:
#             continue
#         if task['estimation_procedure'] != '33% Holdout set':
#             continue
#         if task['did'] != did:
#             continue
#         #if 'evaluation_measures' in task:
#         #    continue
#         cc18_did_to_task_id[task['did']].append(task_id)
# print(len(cc18_did_to_task_id))
# #print(cc18_did_to_task_id)
# print([l[0] for l in list(cc18_did_to_task_id.values())])